First, install dependencies and download the data

In [1]:
#%pip install -r requirements.txt
#!curl -l -o data.csv "https://phl.carto.com/api/v2/sql?q=SELECT+*,+ST_Y(the_geom)+AS+lat,+ST_X(the_geom)+AS+lng+FROM+opa_properties_public&filename=opa_properties_public&format=csv&skipfields=cartodb_id"

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [27]:
df = pd.read_csv('data.csv')

/tmp/ipykernel_6262/1507164070.py:1: DtypeWarning: Columns (5,13,22,26,31,54,55,68,70,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [6]:
df.head()

,the_geom,the_geom_webmercator,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,...,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid,lat,lng
0,0101000020E6100000288FB7AD4BC452C0C6E4F1E75A01...,0101000020110F00002693FA5D94E05FC1EE7D5D356691...,2022-05-24,NaN,120' NW EADOM ST,987458.0,RB,NON PD PKG LOT COMMERCIAL,6,VACANT LAND,...,NaN,NaN,19137.0,IRMX,1001189957,NaN,NaN,433143301,40.010587,-75.067119
1,0101000020E610000015F6916732CB52C0D4B37C0242FE...,0101000020110F0000F30438544DEC5FC1AAE24928F78D...,2022-05-24,NaN,273' N OF NORRIS ST,NaN,SR,VACANT LAND RESIDE < ACRE,6,VACANT LAND,...,NaN,NaN,19121.0,RSA5,1001505175,NaN,NaN,433143302,39.986389,-75.174951
2,0101000020E610000066AC299B48CA52C053CA1DF46CFD...,0101000020110F00004ACDE132C0EA5FC17F6F06010B8D...,2022-05-24,NaN,"263' 8"" N COLUMBIA AVE",NaN,U50,ROW CONV/APT 3 STY MASON,2,MULTI FAMILY,...,1890.0,NaN,19121.0,RM1,1001622123,22,ROW TYPICAL,433143303,39.979887,-75.160682
3,0101000020E6100000000000000000F87F000000000000...,NaN,2022-05-24,NaN,"1,115.962' S PATTISON AVE",NaN,SS,VACANT LAND RESIDE ACRE+,6,VACANT LAND,...,NaN,NaN,NaN,NaN,1001319180,NaN,NaN,433143304,NaN,NaN
4,0101000020E61000000C5CFE385BC452C0E38B05E75601...,0101000020110F0000F7332EC5AEE05FC1240EE0C46191...,2022-05-24,NaN,NEC SCATTERGOOD TO,NaN,RB,NON PD PKG LOT COMMERCIAL,6,VACANT LAND,...,NaN,NaN,19137.0,IRMX,1001189938,NaN,NaN,433143305,40.010465,-75.068068


In [7]:
df.columns

Index(['the_geom', 'the_geom_webmercator', 'assessment_date', 'basements',
       'beginning_point', 'book_and_page', 'building_code',
       'building_code_description', 'category_code',
       'category_code_description', 'census_tract', 'central_air',
       'cross_reference', 'date_exterior_condition', 'depth',
       'exempt_building', 'exempt_land', 'exterior_condition', 'fireplaces',
       'frontage', 'fuel', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'homestead_exemption',
       'house_extension', 'house_number', 'interior_condition', 'location',
       'mailing_address_1', 'mailing_address_2', 'mailing_care_of',
       'mailing_city_state', 'mailing_street', 'mailing_zip', 'market_value',
       'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
       'number_of_rooms', 'number_stories', 'off_street_open',
       'other_building', 'owner_1', 'owner_2', 'parcel_number', 'parcel_shape',
       'quality_grade', 'recording_d

Here we can see all of the columns available in the dataset. Many of these columns are not really necesary. For example, geographical data such as street addresses are not very useful because they don't contain any categorical or numerical data that can be easily consumed by a linear regression mode.

In [67]:
COLUMNS_TO_REMOVE = """
the_geom
the_geom_webmercator
beginning_point
book_and_page
building_code_description
category_code_description
cross_reference
geographic_ward
house_number
location
mailing_address_1
mailing_address_2
mailing_care_of
mailing_city_state
mailing_street
mailing_zip
other_building
owner_1
owner_2
parcel_number
registry_number
state_code
street_code
street_designation
street_direction
street_name
suffix
pin
building_code_description_new
objectid
unit
"""
df_first_column_drop = df.drop(columns=COLUMNS_TO_REMOVE.split())

In [68]:
df_first_column_drop.to_csv('data_first_clean.csv', index=False)

Now that we have removed some of the columns, lets see what we are left with.

In [69]:
df_first_column_drop.dtypes

assessment_date             object
basements                   object
building_code               object
category_code                int64
census_tract               float64
central_air                 object
date_exterior_condition     object
depth                      float64
exempt_building            float64
exempt_land                float64
exterior_condition         float64
fireplaces                 float64
frontage                   float64
fuel                        object
garage_spaces              float64
garage_type                 object
general_construction        object
homestead_exemption          int64
house_extension             object
interior_condition         float64
market_value               float64
market_value_date          float64
number_of_bathrooms        float64
number_of_bedrooms         float64
number_of_rooms            float64
number_stories             float64
off_street_open            float64
parcel_shape                object
quality_grade       

Lets do some digging into these 1 by 1. To start, I will start grouping the features into a few different types:
- DateTime
- Categorical
- Numerical

In [74]:
DATETIME_COLUMNS = """
assessment_date
date_exterior_condition
market_value_date
recording_date
sale_date
year_built
market_value_date
sale_date
"""

CATEGORICAL_COLUMNS = """
basements
building_code
category_code
central_air
fuel
general_construction
garage_type
other_building
parcel_shape
quality_grade
separate_utilities
sewer
site_type
street_direction
topography
type_heater
unfinished
utility
view_type
year_built_estimate
zip_code
zoning
building_code_new
"""

NUMERIC_COLUMNS = """
census_tract
depth
exempt_building
exempt_land
exterior_condition
fireplaces
frontage
garage_spaces
homestead_exemption
house_extension
interior_condition
number_of_bathrooms
number_of_bedrooms
number_of_rooms
number_stories
market_value
off_street_open
taxable_building
taxable_land
total_area
total_livable_area
sale_price
lat
lng
"""

We still have a lot of columns. It is very likely that many more of these will get dropped for the following reasons:
1. Too many missing values
2. Too many unique values
3. Not enough correlation with the target variable

Let's start by dealing with the first case: too many missing values. We will check this by counting the number of missing values in each column and sorting by this number.

There are also a few columns that seem to have a default value put in them instead of being left blank. Lets also change these default values to na, so they can be counted correctly.

In [75]:
df_first_column_drop['depth'] = df_first_column_drop['depth'].replace(0, np.nan)
df_first_column_drop['total_area'] = df_first_column_drop['total_area'].replace(0, np.nan)
df_first_column_drop['total_livable_area'] = df_first_column_drop['total_livable_area'].replace(0, np.nan)
df_first_column_drop['year_built'] = df_first_column_drop['year_built'].replace(0, np.nan)

In [76]:
"""
We still have a lot of columns. It is very likely that many more of these will get dropped for the following reasons:
1. Too many missing values
2. Too many unique values
3. Not enough correlation with the target variable

Let's start by dealing withe the first case: too many missing values. We will check this by counting the number of missing values in each column and sorting by this number.
"""
#print(df_first_column_drop.isna().sum().sort_values(ascending=False))
# Print the number of missing values and the percentage of missing values
missing_values = df_first_column_drop.isna().sum().sort_values(ascending=False)
missing_values = missing_values[missing_values > 0]
missing_values = pd.DataFrame(missing_values, columns=['missing_values'])
missing_values['percentage_missing'] = missing_values['missing_values'] / len(df_first_column_drop)
print(missing_values)

                         missing_values  percentage_missing
market_value_date                582937            1.000000
unfinished                       582866            0.999878
utility                          582774            0.999720
site_type                        580454            0.995741
date_exterior_condition          580279            0.995440
sewer                            568010            0.974393
number_of_rooms                  567929            0.974255
garage_type                      560331            0.961221
house_extension                  556640            0.954889
separate_utilities               554744            0.951636
fuel                             535365            0.918393
central_air                      289516            0.496651
type_heater                      282172            0.484052
basements                        252910            0.433855
year_built_estimate              171840            0.294783
garage_spaces                     84805 

Now some of these pieces of missing data won't be that big of a deal because we can either fill or impute the data. We can also drop rows with missing data, but we should do this sparingly. For some categorical columns, you could set a default value, but I will not be doing this much. You cannot assume exactly what the person who entered the data intended by leaving it blank, and filling it could cause innacuracies, especially with columns with a lot of missing data. But for some columns, there is simply too much missing data. For this reason, I will be dropping all columns with more than 25% missing data.

In [77]:
COLUMNS_TO_REMOVE_MISSING_VALUES = missing_values[missing_values['percentage_missing'] > 0.25].index
df_second_column_drop = df_first_column_drop.drop(columns=COLUMNS_TO_REMOVE_MISSING_VALUES)

In [78]:
df_second_column_drop.to_csv('data_second_clean.csv', index=False)

Now let's recheck the list of columns with missing data, and address each one indiviually.

In [79]:
missing_values_2 = df_second_column_drop.isna().sum().sort_values(ascending=False)
missing_values_2 = missing_values_2[missing_values_2 > 0]
missing_values_2 = pd.DataFrame(missing_values_2, columns=['missing_values'])
missing_values_2['percentage_missing'] = missing_values_2['missing_values'] / len(df_second_column_drop)
print(missing_values_2)

                      missing_values  percentage_missing
garage_spaces                  84805            0.145479
fireplaces                     83568            0.143357
number_of_bathrooms            83271            0.142847
interior_condition             80264            0.137689
exterior_condition             80101            0.137409
number_of_bedrooms             80025            0.137279
number_stories                 79827            0.136939
general_construction           60330            0.103493
quality_grade                  56108            0.096251
building_code_new              45781            0.078535
year_built                     44381            0.076133
total_livable_area             44058            0.075579
topography                     38311            0.065721
depth                          36853            0.063220
total_area                     34376            0.058970
view_type                      20623            0.035378
off_street_open                

Now that we are left with columns with most of their data filled in, it is much safter to start filling in with default or imputed data.

In [84]:
from sklearn.impute import SimpleImputer
median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')
df_fill = df_second_column_drop
# garage_spaces can just be filled with 0. It is a fair assumption that if the value is missing, there is no garage
df_fill['garage_spaces'] = df_fill['garage_spaces'].fillna(0)
# fireplaces can just be filled with 0. It is a fair assumption that if the value is missing, there are no fireplaces
df_fill['fireplaces'] = df_fill['fireplaces'].fillna(0)
# number of bathrooms can be filled with the median
df_fill['number_of_bathrooms'] = median_imputer.fit_transform(df_fill[['number_of_bathrooms']])
# interior condition can be filled with the median
df_fill['interior_condition'] = median_imputer.fit_transform(df_fill[['interior_condition']])
# exterior condition can be filled with the median
df_fill['exterior_condition'] = median_imputer.fit_transform(df_fill[['exterior_condition']])
# number of bedrooms can be filled with the median
df_fill['number_of_bedrooms'] = median_imputer.fit_transform(df_fill[['number_of_bedrooms']])
# number of stories can be filled with the median
df_fill['number_stories'] = median_imputer.fit_transform(df_fill[['number_stories']])
# NOTE: Maybe change these to mode
# general construction can be filled with a new category called 'unknown'
df_fill['general_construction'] = df_fill['general_construction'].fillna('unknown')
# quality grade will be skipped for now because it needs to be transformed into a numeric column
# building code new can be filled with a new category called 'unknown'
df_fill['building_code_new'] = df_fill['building_code_new'].fillna('unknown')
# year built can be filled with the median
df_fill['year_built'] = median_imputer.fit_transform(df_fill[['year_built']])
# total livable area can be filled with the median
df_fill['total_livable_area'] = median_imputer.fit_transform(df_fill[['total_livable_area']])
# topography can be filled with a new category called 'unknown'
df_fill['topography'] = df_fill['topography'].fillna('unknown')
# depth can be filled with the median
df_fill['depth'] = median_imputer.fit_transform(df_fill[['depth']])
# total area can be filled with the median
df_fill['total_area'] = median_imputer.fit_transform(df_fill[['total_area']])
# view type can be filled with a new category called 'unknown'
df_fill['view_type'] = df_fill['view_type'].fillna('unknown')
# off street open can be filled with the median
df_fill['off_street_open'] = median_imputer.fit_transform(df_fill[['off_street_open']])
# assessment date will be skipped for now because it needs to be transformed into a numeric column
# frontage can be filled with the median
df_fill['frontage'] = median_imputer.fit_transform(df_fill[['frontage']])
# recording date will be skipped for now because it needs to be transformed into a numeric column
# parcel shape can be filled with a new category called 'unknown'
df_fill['parcel_shape'] = df_fill['parcel_shape'].fillna('unknown')
# NOTE: Maybe change this to drop
# sale price can be filled with the median
df_fill['sale_price'] = median_imputer.fit_transform(df_fill[['sale_price']])
# sale date will be skipped for now because it needs to be transformed into a numeric column
# zoning can be filled with a new category called 'unknown'
df_fill['zoning'] = df_fill['zoning'].fillna('unknown')
# census tract can be filled with the median
df_fill['census_tract'] = median_imputer.fit_transform(df_fill[['census_tract']])
# NOTE: Maybe change this to mode
# zip code can be filled with a new category called 'unknown'
df_fill['zip_code'] = df_fill['zip_code'].fillna('unknown')
# lat and lng can be filled with the median
df_fill['lat'] = median_imputer.fit_transform(df_fill[['lat']])
df_fill['lng'] = median_imputer.fit_transform(df_fill[['lng']])
# building code can be filled with a new category called 'unknown'
df_fill['building_code'] = df_fill['building_code'].fillna('unknown')
# taxable building can be filled with the median
df_fill['taxable_building'] = median_imputer.fit_transform(df_fill[['taxable_building']])
# exempt land can be filled with the median
df_fill['exempt_land'] = median_imputer.fit_transform(df_fill[['exempt_land']])
# exempt building can be filled with the median
df_fill['exempt_building'] = median_imputer.fit_transform(df_fill[['exempt_building']])
# taxable land can be filled with the median
df_fill['taxable_land'] = median_imputer.fit_transform(df_fill[['taxable_land']])
# NOTE: Maybe change this to drop
# market value can be filled with the median
df_fill['market_value'] = median_imputer.fit_transform(df_fill[['market_value']])

Lets check again our missing data

In [85]:
missing_values_3 = df_fill.isna().sum().sort_values(ascending=False)
missing_values_3 = missing_values_3[missing_values_3 > 0]
missing_values_3 = pd.DataFrame(missing_values_3, columns=['missing_values'])
missing_values_3['percentage_missing'] = missing_values_3['missing_values'] / len(df_second_column_drop)
print(missing_values_3)

                 missing_values  percentage_missing
quality_grade             56108            0.096251
assessment_date            4920            0.008440
recording_date             3775            0.006476
sale_date                  2485            0.004263


So we still need to get rid of the missing data for the date information by first converting them to datetimes, then either imputing or dropping NA values.

In [96]:
df_date_fix = df_fill
# Error from pandas: Out of bounds nanosecond timestamp: 3033-03-03, at position 49425. 
# This is likely a typo. Rather than fixing it, we will just drop the row, so as not to assume the correct date.
df_date_fix = df_date_fix.drop(index=49425)

df_date_fix['assessment_date'] = pd.to_datetime(df_date_fix['assessment_date'])
df_date_fix['recording_date'] = pd.to_datetime(df_date_fix['recording_date'])
df_date_fix['sale_date'] = pd.to_datetime(df_date_fix['sale_date'])

# Then we can convert these columns to numeric
df_date_fix['assessment_date'] = df_date_fix['assessment_date'].astype(int)
df_date_fix['recording_date'] = df_date_fix['recording_date'].astype(int)
df_date_fix['sale_date'] = df_date_fix['sale_date'].astype(int)

# Now we can fill the missing values
df_date_fix['assessment_date'] = median_imputer.fit_transform(df_date_fix[['assessment_date']])
df_date_fix['recording_date'] = median_imputer.fit_transform(df_date_fix[['recording_date']])
df_date_fix['sale_date'] = median_imputer.fit_transform(df_date_fix[['sale_date']])

Now lets take a look